# File Automation

### LIBRARIES and install methods

In [ ]:
!pip install watchdog

In [ ]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os
import time

In [ ]:
DOWNLOADS_FOLDER = os.path.expanduser('~/Downloads')  # Path to the Downloads folder on your PC

### CLASS MyHandler:
- This code defines a class named `MyHandler` which inherits from `FileSystemEventHandler` to handle file system events.

- **`on_created` method**: Activated when a new file is created. It checks to make sure the event is for a file (not a directory). If it's a file event, it calls the `organize_files` method with the path of the newly created file.

- **`organize_files` method**: Organizes individual files into appropriate sub-folders based on their file extensions.
  - It first determines the file's extension and converts it to uppercase (e.g., `.pdf` becomes `PDF`).
  - If a file doesn't have an extension, it assigns it to a default folder named `OTHERS`.
  - It checks if a sub-folder named after the file type (or `OTHERS`) exists in the `DOWNLOADS_FOLDER`. If not, it creates one.
  - Finally, it moves the file into the corresponding sub-folder.

In summary, the code watches for new files being added to a directory. When a new file appears, it organizes that file into a sub-folder based on its file type. If the file doesn't have a recognizable type, it places it in a folder named `OTHERS`.

In [2]:
class MyHandler(FileSystemEventHandler):

    def on_created(self, event):
        if not event.is_directory:  # Ensure it's a file event
            self.organize_files(event.src_path)

    def organize_files(self, file_path):
        # Determine file type (using file extension)
        _, file_extension = os.path.splitext(file_path)
        file_extension = file_extension[1:].upper()  # Convert to uppercase
        
        # Define sub-folder based on file type, e.g., .PDF -> PDF, .jpg -> JPG
        if not file_extension:
            folder_name = 'OTHERS'
        else:
            folder_name = file_extension

        # Create sub-folder in Downloads if it doesn't exist
        destination_folder = os.path.join(DOWNLOADS_FOLDER, folder_name)
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        # Move file into the corresponding sub-folder
        dest_path = os.path.join(destination_folder, os.path.basename(file_path))
        os.rename(file_path, dest_path)

1. An event handler, named `MyHandler`, is initialized. This handler contains the logic to process certain file events, specifically the creation of new files.
2. An observer is set up and configured to watch the "Downloads" folder. It is set to only monitor the top-level directory and not any sub-directories within it.
3. The observer is then started to begin actively monitoring for changes.
4. Before actively monitoring for new files, the code first organizes any existing files in the "Downloads" folder. It does this by looping through each file in the directory, checking if it is indeed a file (and not a sub-directory), and then invoking the `organize_files` method to move it to an appropriate sub-folder based on its type (e.g., PDF files go to a "PDF" sub-folder).
5. After organizing the existing files, the code enters an infinite loop where it waits and lets the observer do its job. Every 10 seconds, it pauses using `time.sleep(10)`. This loop continues until the user interrupts the script (e.g., by pressing `Ctrl+C`).
6. If the user interrupts the script, the observer is stopped to cease its monitoring activity.
7. Finally, `observer.join()` ensures that the observer thread is properly terminated before the script exits.

In [ ]:
# Initialize the observer
event_handler = MyHandler()
observer = Observer()
observer.schedule(event_handler, DOWNLOADS_FOLDER, recursive=False)  # Only monitor the top-level directory, not sub-directories
observer.start()

try:
    # Organize existing files in the Downloads folder before starting to monitor
    for filename in os.listdir(DOWNLOADS_FOLDER):
        file_path = os.path.join(DOWNLOADS_FOLDER, filename)
        if os.path.isfile(file_path):
            event_handler.organize_files(file_path)
    
    while True:
        time.sleep(10)
except KeyboardInterrupt:
    observer.stop()
observer.join()